# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment   groups? 
            * If so, how do you plan to account for this in your experimental design?

In [13]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
print(df.head())
print(df.info())
print("Actions:", df['action'].unique())
print("Groups:", df['group'].unique())
print(f"Dataset shape: {df.shape}")
print(df.isnull().sum())
unique_users = df['id'].nunique()
total_rows = df.shape[0]
print(f"Unique users: {unique_users}")
print(f"Total rows: {total_rows}")
user_counts = df['id'].value_counts()
print(user_counts.value_counts())

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
Actions: ['view' 'click']
Groups: ['experiment' 'control']
Dataset shape: (8188, 4)
timestamp    0
id           0
group        0
action       0
dtype: int64
Unique users: 6328
Total rows: 8188
count
1    4468
2    1860
Name: count, dtype: int64


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [33]:
#Your code here
from statsmodels. stats.proportion import proportions_ztest
user_group_counts = df.groupby("id")["group"].nunique()
overlap_ids = user_group_counts[user_group_counts > 1].index
df_clean = df[~df["id"].isin(overlap_ids)]

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [37]:
#Your code here
user_actions = df_clean.pivot_table(index="id", columns="action", aggfunc="size", fill_value=0).reset_index()
user_group = df_clean[['id', 'group']].drop_duplicates()
merged = pd.merge(user_actions, user_group, on='id')
viewed = merged[merged['view'] > 0]
control = viewed[viewed['group'] == 'control']
experiment = viewed[viewed['group'] == 'experiment']
control_clicks = control['click'].sum()
control_total = len(control)
experiment_clicks = experiment['click'].sum()
experiment_total = len(experiment)
ctr_control = control_clicks / control_total
expected_clicks_exp = experiment_total * ctr_control
print(f"Control CTR: {ctr_control:.4f}")
print(f"Expected Clicks in Experiment Group (under H₀): {expected_clicks_exp:.2f}")



Control CTR: 0.2797
Expected Clicks in Experiment Group (under H₀): 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [41]:
#Your code here
import numpy as np 
std_dev = np.sqrt(experiment_total * ctr_control * (1- ctr_control))
z_score = (experiment_clicks - expected_clicks_exp) / std_dev
print(f"z-score: {z_score:4f}")

z-score: 3.662536


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [44]:
#Your code here
from scipy.stats import norm
p_value = norm.sf(z_score)
print(f"P-value (one-tailed test): {p_value:.4f}")

P-value (one-tailed test): 0.0001


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Yes, the result from the manual calculation closely matches the earlier statistical test. Both show that the difference in click rates isn’t statistically significant, so we can’t confidently say the experimental homepage performed better.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.